# Coalescent simulations

In [ ]:
import msprime
import numpy as np
import matplotlib.pyplot as plt

set up parameters

In [ ]:
L = 1_000_000 # genome size
rho = mu = 1e-8 # mutation rate
n_subpops = 2 # number of subpopulations
subpop_size = 1e4 # size of each subpopulation
migration_rate = 1e-4 # migration rate

simulate ancestry (tree sequnce = ARG) without mutations

In [ ]:
ts_no_mut = msprime.sim_ancestry(
    samples = {f"pop_{i}": 10 for i in range(n_subpops)},
    demography = msprime.Demography.island_model([subpop_size] * n_subpops, migration_rate),
    ploidy = 2,
    sequence_length = L,
    recombination_rate = rho, # cross over recombination
    random_seed=123
)
ts_no_mut

add mutations

In [ ]:
ts_mutated = msprime.sim_mutations(
    ts_no_mut,
    rate = mu,
    random_seed=456
)
ts_mutated

the total size of tree changed. With more mutations, more information

## Analysing patterns of relatedness

In [ ]:
np.random.seed(10)
sample_a = np.random.choice(ts_no_mut.samples(population=0), size=1)[0]
sample_b = np.random.choice(ts_no_mut.samples(population=1), size=1)[0]

av_MRCA = 0

for tree in ts_no_mut.trees():
    # get tMRCA and weight by each tree's span
    av_MRCA += tree.tmrca(sample_a, sample_b) * tree.span / ts_no_mut.sequence_length

print(
    f"Average tMRCA betweeb sample {sample_a} (pop_0) and "
    f"{sample_b} (pop_1) is: {av_MRCA:.2f} {ts_no_mut.time_units}")

In [ ]:
print(
    f"Genetic divergence between samples {sample_a} and {sample_b} is: "
    f"{ts_mutated.divergence([[sample_a], [sample_b]]):.6f}")

In [ ]:
ab_dist = ts_no_mut.divergence([[sample_a], [sample_b]], mode="branch")
ab_dist

In [ ]:
print(f"{ab_dist * mu: .6f}")

In [ ]:
n_reps = 20

ts_reps = list(
    msprime.sim_ancestry(
        samples = {f"pop_{i}": 10 for i in range(n_subpops)},
        demography = msprime.Demography.island_model([subpop_size] * n_subpops, migration_rate),
        ploidy = 2,
        sequence_length = L,
        recombination_rate = rho, # cross over recombination
        random_seed=123,
        num_replicates = n_reps
    )
)

In [ ]:
ts_mutated_reps = [msprime.sim_mutations(ts, rate=mu/100, random_seed=i+4) for i, ts in enumerate(ts_reps)]

In [ ]:
def sample_set(ts):
    return [ts.samples(population=p.id) for p in ts.populations()]

In [ ]:
sample_set(ts_reps[0])

In [ ]:
Fst_genealogy = np.array([ts.Fst(sample_set(ts), mode="branch") for ts in ts_reps])

In [ ]:
Fst_genetic_var = np.array([ts.Fst(sample_set(ts)) for ts in ts_mutated_reps])

In [ ]:
Fst_theory = 1 / (4 * subpop_size * migration_rate * (n_subpops / (n_subpops - 1))**2 + 1)

In [ ]:
plt.scatter(["Genetic variation"] * 20, Fst_genetic_var)
plt.scatter(["Genealogy"] * 20, Fst_genealogy)
plt.xlabel("Basis of estimate")
plt.ylabel("$F_{ST}$\n(20 replicates)")
plt.axhline(y=Fst_theory, color="grey", ls=":")
plt.text(0.5, Fst_theory+0.005, "Theoretical prediction", ha="center")
plt.show()

In [ ]:
np.var(Fst_genealogy), np.var(Fst_genetic_var)

## Distribution of coalescence times for a sample of 2 haploids

In [ ]:
tsList = list(msprime.sim_ancestry(
    2,
    ploidy=1,
    population_size=10_000,
    num_replicates=10_000
))

In [ ]:
tsList[0].draw_svg(y_axis=True)

In [ ]:
tList = [i.first().tmrca(0, 1) for i in tsList]

In [ ]:
plt.hist(tList, bins=100)
plt.show()